In [ ]:
from dotenv import load_dotenv
import os
from hera.workflows import Task, EmptyDirVolume, models, script, Artifact, Parameter, DAG, Steps, Step, NoneArchiveStrategy, Workflow, WorkflowTemplate
from hera.workflows.models import TemplateRef
from hera.shared import global_config

load_dotenv("/home/otto/s1_zarr/.env")

In [ ]:
global_config.host = "https://dev.services.eodc.eu/workflows/"
global_config.namespace = "inca"
global_config.token = os.getenv("argo_token")
global_config.image = "ghcr.io/oscipal/image_zarr:latest"

In [ ]:
@script()
def write_message(message):
    print(message)

In [ ]:
@script()
def echo_message(message):
    print(f"echo of {message}")

In [ ]:
with WorkflowTemplate(
    name="write-message-template",
    entrypoint="workflow"
) as wt:
    with DAG(name="workflow"):
        step1 = write_message(arguments={"message": "Hello hera!"})
        step2 = echo_message(arguments={"message": step1.result})

        step1 >> step2

In [ ]:
wt.create()

In [ ]:
with Workflow(
    generate_name="use-template-",
    entrypoint="workflow"
) as w:
    with DAG(name="workflow"):
        step1 = Task(name="using-template", template_ref=TemplateRef(name="write-message-template", template="workflow"))

In [ ]:
w.create()